# Workflow Orchestration with ghops

Learn how to automate repository management tasks using ghops' powerful workflow system. Create complex automation pipelines that can maintain, update, and deploy your entire repository portfolio.

## Table of Contents
1. [Understanding Workflow Concepts](#concepts)
2. [Workflow YAML Syntax](#yaml-syntax)
3. [Creating Your First Workflow](#first-workflow)
4. [Building a Morning Maintenance Routine](#morning-routine)
5. [Implementing a Release Pipeline](#release-pipeline)
6. [Conditional Execution](#conditional)
7. [Parallel and Sequential Steps](#parallel)
8. [Monitoring and Debugging](#monitoring)
9. [Advanced Workflow Patterns](#advanced)
10. [Exercises](#exercises)

## Setup and Imports

In [ ]:
import subprocess
import json
import yaml
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import tempfile
import os
import time
from datetime import datetime, timedelta
from IPython.display import display, HTML, clear_output
import networkx as nx

# Helper functions
def run_command(cmd):
    """Run shell command and return output"""
    result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
    return result.stdout, result.stderr, result.returncode

def parse_jsonl(output):
    """Parse JSONL output into list of dicts"""
    results = []
    for line in output.strip().split('\n'):
        if line:
            try:
                results.append(json.loads(line))
            except json.JSONDecodeError:
                continue
    return results

# Create workspace
workspace = tempfile.mkdtemp(prefix="ghops_workflow_")
workflows_dir = Path(workspace) / "workflows"
workflows_dir.mkdir(exist_ok=True)

print(f"Workspace: {workspace}")
print(f"Workflows directory: {workflows_dir}")
print("Setup complete!")

## 1. Understanding Workflow Concepts {#concepts}

ghops workflows are automation blueprints that:
- **Orchestrate** multiple repository operations
- **Schedule** recurring maintenance tasks
- **Automate** complex deployment pipelines
- **React** to repository changes and events

### Key Components

1. **Triggers**: When the workflow runs (schedule, event, manual)
2. **Steps**: Individual actions to perform
3. **Conditions**: Control flow based on state
4. **Variables**: Dynamic values and configuration
5. **Outputs**: Results and artifacts from the workflow

In [ ]:
# Workflow concept visualization
plt.figure(figsize=(12, 8))

# Create a directed graph representing workflow concepts
G = nx.DiGraph()

# Add nodes
nodes = [
    ("Trigger", {"type": "start", "color": "lightgreen"}),
    ("Condition", {"type": "decision", "color": "yellow"}),
    ("Step 1", {"type": "action", "color": "lightblue"}),
    ("Step 2", {"type": "action", "color": "lightblue"}),
    ("Parallel A", {"type": "action", "color": "lightcoral"}),
    ("Parallel B", {"type": "action", "color": "lightcoral"}),
    ("Output", {"type": "end", "color": "lightgray"})
]

for node, attrs in nodes:
    G.add_node(node, **attrs)

# Add edges
edges = [
    ("Trigger", "Condition"),
    ("Condition", "Step 1"),
    ("Condition", "Step 2"),
    ("Step 1", "Parallel A"),
    ("Step 1", "Parallel B"),
    ("Step 2", "Output"),
    ("Parallel A", "Output"),
    ("Parallel B", "Output")
]
G.add_edges_from(edges)

# Layout and draw
pos = nx.spring_layout(G, k=2, iterations=50)
node_colors = [G.nodes[node]['color'] for node in G.nodes()]

nx.draw(G, pos, with_labels=True, node_color=node_colors, 
        node_size=2000, font_size=10, font_weight='bold',
        arrows=True, arrowsize=20, edge_color='gray',
        linewidths=2, edgecolors='black')

plt.title("Workflow Component Flow", fontsize=14, fontweight='bold')
plt.axis('off')
plt.tight_layout()
plt.show()

print("Workflow Components:")
print("• Trigger: Initiates the workflow")
print("• Condition: Determines execution path")
print("• Steps: Sequential actions")
print("• Parallel: Concurrent execution")
print("• Output: Results and artifacts")

## 2. Workflow YAML Syntax {#yaml-syntax}

Workflows are defined in YAML format. Let's explore the syntax:

In [ ]:
# Example workflow structure
example_workflow = {
    "name": "example-workflow",
    "description": "Demonstrates workflow syntax",
    "version": "1.0.0",
    
    "triggers": [
        {"type": "schedule", "cron": "0 9 * * *"},  # Daily at 9 AM
        {"type": "manual"}  # Can be triggered manually
    ],
    
    "variables": {
        "repo_dir": "~/projects",
        "max_age_days": 7,
        "notification_email": "dev@example.com"
    },
    
    "steps": [
        {
            "name": "scan-repos",
            "action": "ghops.list",
            "params": {
                "path": "${repo_dir}",
                "recursive": True
            }
        },
        {
            "name": "check-status",
            "action": "ghops.status",
            "foreach": "${scan-repos.output}",
            "params": {
                "path": "${item.path}"
            }
        },
        {
            "name": "filter-dirty",
            "action": "filter",
            "input": "${check-status.output}",
            "condition": "item.status.clean == false"
        },
        {
            "name": "notify",
            "action": "email",
            "condition": "${filter-dirty.output.length} > 0",
            "params": {
                "to": "${notification_email}",
                "subject": "Repositories need attention",
                "body": "Found ${filter-dirty.output.length} repositories with uncommitted changes"
            }
        }
    ],
    
    "outputs": {
        "dirty_repos": "${filter-dirty.output}",
        "total_scanned": "${scan-repos.output.length}"
    }
}

# Save as YAML
workflow_file = workflows_dir / "example-workflow.yaml"
with open(workflow_file, 'w') as f:
    yaml.dump(example_workflow, f, default_flow_style=False)

print("Example Workflow YAML:")
print("=" * 60)
print(yaml.dump(example_workflow, default_flow_style=False))

## 3. Creating Your First Workflow {#first-workflow}

Let's create a simple workflow that checks repository health:

In [ ]:
# Create sample repositories for testing
repos_dir = Path(workspace) / "repos"
repos_dir.mkdir(exist_ok=True)

def create_test_repo(name, has_changes=False, has_remote=False):
    """Create a test repository with specific characteristics"""
    repo_path = repos_dir / name
    repo_path.mkdir(exist_ok=True)
    
    os.chdir(repo_path)
    run_command("git init")
    run_command("git config user.email 'test@example.com'")
    run_command("git config user.name 'Test User'")
    
    # Create initial files
    (repo_path / "README.md").write_text(f"# {name}")
    run_command("git add .")
    run_command("git commit -m 'Initial commit'")
    
    if has_changes:
        (repo_path / "new_file.txt").write_text("Uncommitted changes")
    
    if has_remote:
        run_command(f"git remote add origin https://github.com/test/{name}.git")
    
    return repo_path

# Create test repositories
test_repos = [
    create_test_repo("healthy-repo", False, True),
    create_test_repo("dirty-repo", True, True),
    create_test_repo("no-remote-repo", False, False),
    create_test_repo("abandoned-repo", True, False)
]

print(f"Created {len(test_repos)} test repositories:")
for repo in test_repos:
    print(f"  - {repo.name}")

In [ ]:
# Create health check workflow
health_check_workflow = {
    "name": "repository-health-check",
    "description": "Check health of all repositories",
    "version": "1.0.0",
    
    "steps": [
        {
            "name": "discover",
            "action": "ghops.list",
            "params": {
                "path": str(repos_dir)
            }
        },
        {
            "name": "analyze",
            "action": "ghops.status",
            "foreach": "${discover.output}",
            "params": {
                "path": "${item.path}"
            }
        },
        {
            "name": "categorize",
            "action": "custom.categorize_health",
            "input": "${analyze.output}"
        },
        {
            "name": "report",
            "action": "custom.generate_report",
            "params": {
                "categories": "${categorize.output}",
                "format": "markdown"
            }
        }
    ],
    
    "outputs": {
        "health_report": "${report.output}",
        "unhealthy_repos": "${categorize.output.unhealthy}"
    }
}

# Save workflow
health_workflow_file = workflows_dir / "health-check.yaml"
with open(health_workflow_file, 'w') as f:
    yaml.dump(health_check_workflow, f, default_flow_style=False)

print("Health Check Workflow created!")
print(f"Location: {health_workflow_file}")

In [ ]:
# Simulate workflow execution
def simulate_workflow_execution(workflow, context={}):
    """Simulate executing a workflow"""
    print(f"\n🚀 Executing Workflow: {workflow['name']}")
    print("=" * 60)
    
    results = {}
    
    for step in workflow['steps']:
        print(f"\n📌 Step: {step['name']}")
        print(f"   Action: {step['action']}")
        
        # Simulate different actions
        if step['action'] == 'ghops.list':
            # Simulate listing repositories
            stdout, _, _ = run_command(f"ghops list {step['params']['path']}")
            if stdout:
                results[step['name']] = {'output': parse_jsonl(stdout)}
            else:
                # Mock data if command not available
                results[step['name']] = {'output': [
                    {'name': 'healthy-repo', 'path': str(repos_dir / 'healthy-repo')},
                    {'name': 'dirty-repo', 'path': str(repos_dir / 'dirty-repo')},
                    {'name': 'no-remote-repo', 'path': str(repos_dir / 'no-remote-repo')},
                    {'name': 'abandoned-repo', 'path': str(repos_dir / 'abandoned-repo')}
                ]}
            print(f"   Found {len(results[step['name']]['output'])} repositories")
            
        elif step['action'] == 'ghops.status':
            # Simulate status check
            statuses = []
            if 'foreach' in step:
                # Get items from previous step
                prev_step = step['foreach'].replace('${', '').replace('.output}', '')
                items = results.get(prev_step, {}).get('output', [])
                
                for item in items:
                    # Mock status based on repo name
                    status = {
                        'name': item['name'],
                        'path': item['path'],
                        'status': {
                            'clean': 'dirty' not in item['name'] and 'abandoned' not in item['name'],
                            'has_upstream': 'no-remote' not in item['name'] and 'abandoned' not in item['name'],
                            'branch': 'main'
                        }
                    }
                    statuses.append(status)
            
            results[step['name']] = {'output': statuses}
            print(f"   Checked status of {len(statuses)} repositories")
            
        elif step['action'] == 'custom.categorize_health':
            # Categorize repositories by health
            prev_step = step['input'].replace('${', '').replace('.output}', '')
            repos = results.get(prev_step, {}).get('output', [])
            
            categories = {
                'healthy': [],
                'needs_attention': [],
                'unhealthy': []
            }
            
            for repo in repos:
                if repo['status']['clean'] and repo['status']['has_upstream']:
                    categories['healthy'].append(repo['name'])
                elif not repo['status']['clean']:
                    categories['unhealthy'].append(repo['name'])
                else:
                    categories['needs_attention'].append(repo['name'])
            
            results[step['name']] = {'output': categories}
            print(f"   Categorized: {len(categories['healthy'])} healthy, "
                  f"{len(categories['needs_attention'])} need attention, "
                  f"{len(categories['unhealthy'])} unhealthy")
            
        elif step['action'] == 'custom.generate_report':
            # Generate report
            categories = results.get('categorize', {}).get('output', {})
            
            report = "# Repository Health Report\n\n"
            report += f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n"
            
            report += "## Summary\n"
            report += f"- Healthy: {len(categories.get('healthy', []))}\n"
            report += f"- Need Attention: {len(categories.get('needs_attention', []))}\n"
            report += f"- Unhealthy: {len(categories.get('unhealthy', []))}\n\n"
            
            for category, repos in categories.items():
                report += f"## {category.replace('_', ' ').title()}\n"
                for repo in repos:
                    report += f"- {repo}\n"
                report += "\n"
            
            results[step['name']] = {'output': report}
            print(f"   Generated report ({len(report)} characters)")
    
    # Process outputs
    if 'outputs' in workflow:
        print("\n📊 Workflow Outputs:")
        for key, value in workflow['outputs'].items():
            print(f"   {key}: {value}")
    
    print("\n✅ Workflow completed successfully!")
    return results

# Execute the workflow
results = simulate_workflow_execution(health_check_workflow)

In [ ]:
# Display the health report
if 'report' in results and 'output' in results['report']:
    from IPython.display import Markdown
    display(Markdown(results['report']['output']))

## 4. Building a Morning Maintenance Routine {#morning-routine}

Create a comprehensive morning maintenance workflow that runs daily:

In [ ]:
morning_routine = {
    "name": "morning-maintenance",
    "description": "Daily morning repository maintenance routine",
    "version": "2.0.0",
    
    "triggers": [
        {"type": "schedule", "cron": "0 6 * * 1-5"},  # 6 AM weekdays
    ],
    
    "variables": {
        "workspace": "~/projects",
        "backup_dir": "~/backups/repos",
        "slack_webhook": "${SLACK_WEBHOOK_URL}",
        "auto_commit": False,
        "auto_push": False
    },
    
    "steps": [
        # Phase 1: Discovery and Analysis
        {
            "name": "discover_repos",
            "action": "ghops.list",
            "params": {
                "path": "${workspace}",
                "recursive": True
            }
        },
        
        # Phase 2: Update and Sync
        {
            "name": "pull_updates",
            "action": "git.pull",
            "foreach": "${discover_repos.output}",
            "condition": "item.remote.url != null",
            "params": {
                "path": "${item.path}",
                "rebase": True
            },
            "on_error": "continue"  # Don't stop on pull errors
        },
        
        # Phase 3: Check Status
        {
            "name": "check_status",
            "action": "ghops.status",
            "foreach": "${discover_repos.output}",
            "params": {
                "path": "${item.path}"
            }
        },
        
        # Phase 4: Clean up
        {
            "name": "cleanup_branches",
            "action": "git.prune",
            "foreach": "${discover_repos.output}",
            "params": {
                "path": "${item.path}",
                "remote": True,
                "merged": True
            }
        },
        
        # Phase 5: Security Scan
        {
            "name": "security_scan",
            "action": "security.scan",
            "foreach": "${discover_repos.output}",
            "params": {
                "path": "${item.path}",
                "check_secrets": True,
                "check_vulnerabilities": True
            }
        },
        
        # Phase 6: Backup Critical Repos
        {
            "name": "backup",
            "action": "backup.create",
            "foreach": "${discover_repos.output}",
            "condition": "item.tags.contains('critical')",
            "params": {
                "source": "${item.path}",
                "destination": "${backup_dir}/${item.name}",
                "compress": True
            }
        },
        
        # Phase 7: Generate Reports
        {
            "name": "generate_summary",
            "action": "report.generate",
            "params": {
                "status_data": "${check_status.output}",
                "security_data": "${security_scan.output}",
                "format": "html"
            }
        },
        
        # Phase 8: Notifications
        {
            "name": "notify_issues",
            "action": "slack.send",
            "condition": "${security_scan.output.issues.length} > 0",
            "params": {
                "webhook": "${slack_webhook}",
                "message": "⚠️ Security issues found in ${security_scan.output.issues.length} repositories",
                "attachments": "${generate_summary.output}"
            }
        }
    ],
    
    "outputs": {
        "total_repos": "${discover_repos.output.length}",
        "repos_with_issues": "${security_scan.output.issues}",
        "summary_report": "${generate_summary.output}"
    },
    
    "on_failure": {
        "action": "email.send",
        "params": {
            "to": "admin@example.com",
            "subject": "Morning maintenance workflow failed",
            "body": "Error details: ${error.message}"
        }
    }
}

# Save workflow
morning_workflow_file = workflows_dir / "morning-maintenance.yaml"
with open(morning_workflow_file, 'w') as f:
    yaml.dump(morning_routine, f, default_flow_style=False)

print("Morning Maintenance Workflow:")
print("=" * 60)
print(f"Steps: {len(morning_routine['steps'])}")
for step in morning_routine['steps']:
    print(f"  • {step['name']}: {step['action']}")

In [ ]:
# Visualize morning routine workflow
plt.figure(figsize=(14, 10))

# Create workflow graph
G = nx.DiGraph()

# Add nodes for each step
phases = [
    ("Start", "Discovery", ["discover_repos"]),
    ("Discovery", "Update", ["pull_updates"]),
    ("Update", "Analysis", ["check_status"]),
    ("Analysis", "Cleanup", ["cleanup_branches"]),
    ("Cleanup", "Security", ["security_scan"]),
    ("Security", "Backup", ["backup"]),
    ("Backup", "Report", ["generate_summary"]),
    ("Report", "Notify", ["notify_issues"]),
    ("Notify", "End", [])
]

# Add phase nodes
for from_phase, to_phase, steps in phases:
    G.add_edge(from_phase, to_phase)

# Layout and draw
pos = nx.spring_layout(G, k=3, iterations=50)

# Draw nodes with different colors for different phases
node_colors = ['lightgreen' if node == 'Start' else 
               'lightcoral' if node == 'End' else 
               'lightblue' for node in G.nodes()]

nx.draw(G, pos, with_labels=True, node_color=node_colors,
        node_size=2500, font_size=10, font_weight='bold',
        arrows=True, arrowsize=20, edge_color='gray',
        linewidths=2, edgecolors='black')

# Add step details
step_text = "\n".join([f"{i+1}. {step['name']}" for i, step in enumerate(morning_routine['steps'])])
plt.text(1.2, 0.5, step_text, fontsize=8, 
         bbox=dict(boxstyle="round,pad=0.5", facecolor="lightyellow"))

plt.title("Morning Maintenance Workflow Phases", fontsize=14, fontweight='bold')
plt.axis('off')
plt.tight_layout()
plt.show()

## 5. Implementing a Release Pipeline {#release-pipeline}

Create a sophisticated release pipeline workflow:

In [ ]:
release_pipeline = {
    "name": "release-pipeline",
    "description": "Automated release pipeline for projects",
    "version": "3.0.0",
    
    "parameters": {
        "project_path": {"type": "string", "required": True},
        "version": {"type": "string", "required": True},
        "release_type": {"type": "string", "enum": ["major", "minor", "patch"]},
        "dry_run": {"type": "boolean", "default": False}
    },
    
    "stages": [
        {
            "name": "pre-release",
            "steps": [
                {
                    "name": "validate_version",
                    "action": "version.validate",
                    "params": {
                        "version": "${version}",
                        "current": "${git.tag.latest}"
                    }
                },
                {
                    "name": "run_tests",
                    "action": "test.run",
                    "params": {
                        "path": "${project_path}",
                        "coverage": True,
                        "min_coverage": 80
                    }
                },
                {
                    "name": "lint_code",
                    "action": "lint.run",
                    "params": {
                        "path": "${project_path}",
                        "fix": True
                    }
                },
                {
                    "name": "security_audit",
                    "action": "security.audit",
                    "params": {
                        "path": "${project_path}",
                        "fail_on_high": True
                    }
                }
            ]
        },
        {
            "name": "build",
            "steps": [
                {
                    "name": "update_version",
                    "action": "version.update",
                    "params": {
                        "path": "${project_path}",
                        "version": "${version}",
                        "files": ["setup.py", "package.json", "Cargo.toml"]
                    }
                },
                {
                    "name": "build_artifacts",
                    "action": "build.create",
                    "params": {
                        "path": "${project_path}",
                        "output": "./dist"
                    }
                },
                {
                    "name": "generate_changelog",
                    "action": "changelog.generate",
                    "params": {
                        "from": "${git.tag.latest}",
                        "to": "HEAD",
                        "format": "markdown"
                    }
                }
            ]
        },
        {
            "name": "release",
            "condition": "${dry_run} == false",
            "steps": [
                {
                    "name": "create_tag",
                    "action": "git.tag",
                    "params": {
                        "name": "v${version}",
                        "message": "Release version ${version}",
                        "sign": True
                    }
                },
                {
                    "name": "push_tag",
                    "action": "git.push",
                    "params": {
                        "tags": True
                    }
                },
                {
                    "name": "create_github_release",
                    "action": "github.release",
                    "params": {
                        "tag": "v${version}",
                        "name": "Release ${version}",
                        "body": "${generate_changelog.output}",
                        "artifacts": "${build_artifacts.output}",
                        "prerelease": "${release_type == 'patch'}"
                    }
                }
            ]
        },
        {
            "name": "deploy",
            "condition": "${dry_run} == false",
            "parallel": True,
            "steps": [
                {
                    "name": "publish_pypi",
                    "action": "pypi.publish",
                    "condition": "${project.type} == 'python'",
                    "params": {
                        "artifacts": "${build_artifacts.output}"
                    }
                },
                {
                    "name": "publish_npm",
                    "action": "npm.publish",
                    "condition": "${project.type} == 'javascript'",
                    "params": {
                        "artifacts": "${build_artifacts.output}"
                    }
                },
                {
                    "name": "deploy_docs",
                    "action": "docs.deploy",
                    "params": {
                        "source": "./docs",
                        "target": "gh-pages"
                    }
                }
            ]
        },
        {
            "name": "post-release",
            "steps": [
                {
                    "name": "announce",
                    "action": "social.post",
                    "params": {
                        "platforms": ["twitter", "linkedin"],
                        "message": "🚀 Released version ${version} of ${project.name}! Check out the changelog: ${create_github_release.url}"
                    }
                },
                {
                    "name": "cleanup",
                    "action": "cleanup.artifacts",
                    "params": {
                        "path": "./dist"
                    }
                }
            ]
        }
    ],
    
    "rollback": {
        "enabled": True,
        "on_failure": [
            {
                "action": "git.reset",
                "params": {"hard": True, "to": "${git.commit.before}"}
            },
            {
                "action": "notification.send",
                "params": {
                    "message": "Release ${version} failed and was rolled back"
                }
            }
        ]
    }
}

# Save workflow
release_workflow_file = workflows_dir / "release-pipeline.yaml"
with open(release_workflow_file, 'w') as f:
    yaml.dump(release_pipeline, f, default_flow_style=False)

print("Release Pipeline Stages:")
print("=" * 60)
for stage in release_pipeline['stages']:
    print(f"\n📦 Stage: {stage['name']}")
    if 'condition' in stage:
        print(f"   Condition: {stage['condition']}")
    if stage.get('parallel'):
        print(f"   Execution: Parallel")
    for step in stage['steps']:
        print(f"   - {step['name']}: {step['action']}")

## 6. Conditional Execution {#conditional}

Learn how to use conditions to control workflow execution:

In [ ]:
# Conditional workflow example
conditional_workflow = {
    "name": "smart-deployment",
    "description": "Deploy based on conditions",
    
    "variables": {
        "environment": "${ENV:-development}",
        "auto_deploy": True,
        "min_test_coverage": 75
    },
    
    "steps": [
        {
            "name": "check_branch",
            "action": "git.current_branch",
            "output": "current_branch"
        },
        {
            "name": "run_tests",
            "action": "test.run",
            "output": "test_results"
        },
        {
            "name": "deploy_staging",
            "action": "deploy.staging",
            "condition": "${current_branch} == 'develop' AND ${test_results.passed} == true"
        },
        {
            "name": "deploy_production",
            "action": "deploy.production",
            "condition": "${current_branch} == 'main' AND ${test_results.coverage} >= ${min_test_coverage} AND ${auto_deploy} == true"
        },
        {
            "name": "notify_failure",
            "action": "notify.team",
            "condition": "${test_results.passed} == false OR ${test_results.coverage} < ${min_test_coverage}",
            "params": {
                "message": "Deployment blocked: Tests ${test_results.passed ? 'passed' : 'failed'}, Coverage: ${test_results.coverage}%"
            }
        }
    ]
}

# Demonstrate condition evaluation
test_scenarios = [
    {"branch": "main", "passed": True, "coverage": 85},
    {"branch": "main", "passed": True, "coverage": 70},
    {"branch": "develop", "passed": True, "coverage": 60},
    {"branch": "feature", "passed": False, "coverage": 50}
]

print("Conditional Execution Scenarios:")
print("=" * 70)
print(f"{'Branch':<12} {'Tests':<8} {'Coverage':<10} {'Staging':<10} {'Production':<12} {'Notify':<8}")
print("-" * 70)

for scenario in test_scenarios:
    # Evaluate conditions
    deploy_staging = scenario['branch'] == 'develop' and scenario['passed']
    deploy_production = scenario['branch'] == 'main' and scenario['coverage'] >= 75 and scenario['passed']
    notify = not scenario['passed'] or scenario['coverage'] < 75
    
    print(f"{scenario['branch']:<12} "
          f"{'✓' if scenario['passed'] else '✗':<8} "
          f"{scenario['coverage']}%{'':<7} "
          f"{'Deploy' if deploy_staging else '-':<10} "
          f"{'Deploy' if deploy_production else '-':<12} "
          f"{'Alert' if notify else '-':<8}")

## 7. Parallel and Sequential Steps {#parallel}

Understanding execution modes for optimal performance:

In [ ]:
import asyncio
import random

# Simulate task execution
async def simulate_task(name, duration):
    """Simulate a task that takes some time"""
    print(f"  ▶ Starting: {name}")
    await asyncio.sleep(duration)
    print(f"  ✓ Completed: {name} ({duration}s)")
    return f"{name}_result"

async def run_sequential(tasks):
    """Run tasks sequentially"""
    print("\n🔄 Sequential Execution:")
    print("=" * 40)
    start = time.time()
    
    results = []
    for task_name, duration in tasks:
        result = await simulate_task(task_name, duration)
        results.append(result)
    
    elapsed = time.time() - start
    print(f"\nTotal time: {elapsed:.1f}s")
    return results

async def run_parallel(tasks):
    """Run tasks in parallel"""
    print("\n⚡ Parallel Execution:")
    print("=" * 40)
    start = time.time()
    
    # Create async tasks
    async_tasks = [simulate_task(name, duration) for name, duration in tasks]
    results = await asyncio.gather(*async_tasks)
    
    elapsed = time.time() - start
    print(f"\nTotal time: {elapsed:.1f}s")
    return results

# Define tasks with execution times
tasks = [
    ("Lint Code", 2),
    ("Run Tests", 3),
    ("Build Docker", 4),
    ("Security Scan", 2),
    ("Generate Docs", 1)
]

# Run comparison
print("Execution Mode Comparison")
print("=" * 50)

# Note: In Jupyter, we need to handle async differently
import nest_asyncio
nest_asyncio.apply()

# Sequential execution
loop = asyncio.get_event_loop()
seq_results = loop.run_until_complete(run_sequential(tasks))

# Parallel execution
par_results = loop.run_until_complete(run_parallel(tasks))

In [ ]:
# Visualize execution timeline
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 6))

# Sequential timeline
y_pos = 0
current_time = 0
colors = plt.cm.Set3(range(len(tasks)))

for i, (name, duration) in enumerate(tasks):
    ax1.barh(y_pos, duration, left=current_time, height=0.8, 
            color=colors[i], label=name)
    ax1.text(current_time + duration/2, y_pos, name, 
            ha='center', va='center', fontsize=9)
    current_time += duration

ax1.set_ylim(-0.5, 0.5)
ax1.set_xlim(0, current_time + 1)
ax1.set_xlabel('Time (seconds)')
ax1.set_title('Sequential Execution Timeline')
ax1.set_yticks([])
ax1.grid(True, alpha=0.3)

# Parallel timeline
for i, (name, duration) in enumerate(tasks):
    ax2.barh(i, duration, height=0.8, color=colors[i], label=name)
    ax2.text(duration/2, i, name, ha='center', va='center', fontsize=9)

ax2.set_ylim(-0.5, len(tasks) - 0.5)
ax2.set_xlim(0, max([d for _, d in tasks]) + 1)
ax2.set_xlabel('Time (seconds)')
ax2.set_ylabel('Task')
ax2.set_title('Parallel Execution Timeline')
ax2.set_yticks([])
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Performance comparison
seq_time = sum(d for _, d in tasks)
par_time = max(d for _, d in tasks)
speedup = seq_time / par_time

print(f"\nPerformance Analysis:")
print(f"  Sequential time: {seq_time}s")
print(f"  Parallel time: {par_time}s")
print(f"  Speedup: {speedup:.1f}x")
print(f"  Efficiency: {speedup/len(tasks)*100:.1f}%")

## 8. Monitoring and Debugging {#monitoring}

Learn how to monitor workflow execution and debug issues:

In [ ]:
# Workflow execution monitor
class WorkflowMonitor:
    def __init__(self, workflow_name):
        self.workflow_name = workflow_name
        self.start_time = None
        self.steps = []
        self.current_step = None
        
    def start(self):
        self.start_time = datetime.now()
        print(f"🚀 Workflow '{self.workflow_name}' started at {self.start_time.strftime('%H:%M:%S')}")
    
    def step_start(self, step_name):
        self.current_step = {
            'name': step_name,
            'start': datetime.now(),
            'status': 'running'
        }
        print(f"  ▶ Step '{step_name}' started")
    
    def step_complete(self, status='success', error=None):
        if self.current_step:
            self.current_step['end'] = datetime.now()
            self.current_step['duration'] = (self.current_step['end'] - self.current_step['start']).total_seconds()
            self.current_step['status'] = status
            self.current_step['error'] = error
            
            icon = '✓' if status == 'success' else '✗'
            print(f"  {icon} Step '{self.current_step['name']}' {status} ({self.current_step['duration']:.1f}s)")
            
            if error:
                print(f"    Error: {error}")
            
            self.steps.append(self.current_step)
            self.current_step = None
    
    def complete(self):
        end_time = datetime.now()
        total_duration = (end_time - self.start_time).total_seconds()
        
        successful = sum(1 for s in self.steps if s['status'] == 'success')
        failed = sum(1 for s in self.steps if s['status'] == 'failed')
        
        print(f"\n✅ Workflow completed in {total_duration:.1f}s")
        print(f"   Steps: {successful} successful, {failed} failed")
        
        return self.generate_report()
    
    def generate_report(self):
        report = {
            'workflow': self.workflow_name,
            'start_time': self.start_time.isoformat(),
            'total_duration': (datetime.now() - self.start_time).total_seconds(),
            'steps': self.steps,
            'summary': {
                'total': len(self.steps),
                'successful': sum(1 for s in self.steps if s['status'] == 'success'),
                'failed': sum(1 for s in self.steps if s['status'] == 'failed'),
                'average_duration': sum(s['duration'] for s in self.steps) / len(self.steps) if self.steps else 0
            }
        }
        return report

# Simulate monitored workflow execution
monitor = WorkflowMonitor("test-workflow")
monitor.start()

# Simulate steps
steps_to_execute = [
    ("Initialize", 0.5, "success", None),
    ("Validate Input", 0.3, "success", None),
    ("Process Data", 1.2, "success", None),
    ("External API Call", 2.0, "failed", "Connection timeout"),
    ("Cleanup", 0.5, "success", None)
]

for step_name, duration, status, error in steps_to_execute:
    monitor.step_start(step_name)
    time.sleep(duration)  # Simulate work
    monitor.step_complete(status, error)

# Complete and get report
report = monitor.complete()

In [ ]:
# Visualize workflow execution report
if report:
    # Create DataFrame from steps
    steps_df = pd.DataFrame(report['steps'])
    
    fig, axes = plt.subplots(2, 2, figsize=(14, 8))
    
    # Step duration chart
    ax1 = axes[0, 0]
    colors = ['green' if s == 'success' else 'red' for s in steps_df['status']]
    ax1.bar(range(len(steps_df)), steps_df['duration'], color=colors)
    ax1.set_xticks(range(len(steps_df)))
    ax1.set_xticklabels(steps_df['name'], rotation=45, ha='right')
    ax1.set_ylabel('Duration (seconds)')
    ax1.set_title('Step Execution Times')
    ax1.grid(True, alpha=0.3)
    
    # Success/Failure pie chart
    ax2 = axes[0, 1]
    status_counts = steps_df['status'].value_counts()
    colors = ['#2ecc71' if s == 'success' else '#e74c3c' for s in status_counts.index]
    ax2.pie(status_counts.values, labels=status_counts.index, autopct='%1.0f%%',
           colors=colors, startangle=90)
    ax2.set_title('Step Status Distribution')
    
    # Timeline visualization
    ax3 = axes[1, 0]
    for i, row in steps_df.iterrows():
        color = 'green' if row['status'] == 'success' else 'red'
        ax3.barh(i, row['duration'], left=i*0.5, color=color, alpha=0.7)
        ax3.text(i*0.5 + row['duration']/2, i, row['name'], 
                ha='center', va='center', fontsize=8)
    ax3.set_xlabel('Time (relative)')
    ax3.set_ylabel('Step')
    ax3.set_title('Execution Timeline')
    ax3.set_yticks([])
    
    # Summary text
    ax4 = axes[1, 1]
    ax4.axis('off')
    summary_text = f"""
    Workflow Summary
    ================
    Total Steps: {report['summary']['total']}
    Successful: {report['summary']['successful']}
    Failed: {report['summary']['failed']}
    
    Total Duration: {report['total_duration']:.1f}s
    Avg Step Duration: {report['summary']['average_duration']:.1f}s
    
    Status: {'⚠️ Partial Failure' if report['summary']['failed'] > 0 else '✅ Success'}
    """
    ax4.text(0.1, 0.5, summary_text, fontsize=11, family='monospace',
            bbox=dict(boxstyle="round,pad=0.5", facecolor="lightyellow"))
    
    plt.suptitle(f"Workflow Execution Report: {report['workflow']}", fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

## 9. Advanced Workflow Patterns {#advanced}

Explore advanced patterns for complex automation:

In [ ]:
# Advanced workflow patterns
advanced_patterns = {
    "retry_pattern": {
        "description": "Retry failed operations with exponential backoff",
        "example": {
            "name": "api_call_with_retry",
            "action": "http.request",
            "retry": {
                "max_attempts": 3,
                "backoff": "exponential",
                "initial_delay": 1,
                "max_delay": 30
            }
        }
    },
    
    "circuit_breaker": {
        "description": "Prevent cascading failures",
        "example": {
            "name": "protected_service_call",
            "action": "service.call",
            "circuit_breaker": {
                "failure_threshold": 5,
                "timeout": 60,
                "half_open_requests": 3
            }
        }
    },
    
    "fan_out_fan_in": {
        "description": "Process items in parallel then aggregate results",
        "example": {
            "fan_out": {
                "name": "process_items",
                "action": "process",
                "foreach": "${items}",
                "parallel": True,
                "max_concurrent": 5
            },
            "fan_in": {
                "name": "aggregate_results",
                "action": "aggregate",
                "input": "${process_items.results}"
            }
        }
    },
    
    "saga_pattern": {
        "description": "Distributed transaction with compensating actions",
        "example": {
            "steps": [
                {
                    "name": "reserve_inventory",
                    "action": "inventory.reserve",
                    "compensate": "inventory.release"
                },
                {
                    "name": "charge_payment",
                    "action": "payment.charge",
                    "compensate": "payment.refund"
                },
                {
                    "name": "ship_order",
                    "action": "shipping.create",
                    "compensate": "shipping.cancel"
                }
            ]
        }
    }
}

print("Advanced Workflow Patterns:")
print("=" * 60)

for pattern_name, pattern_info in advanced_patterns.items():
    print(f"\n📋 {pattern_name.replace('_', ' ').title()}")
    print(f"   {pattern_info['description']}")
    print(f"\n   Example:")
    print("   " + yaml.dump(pattern_info['example'], default_flow_style=False).replace('\n', '\n   '))

## 10. Exercises {#exercises}

Practice creating and managing workflows:

### Exercise 1: Create a Backup Workflow
Design a workflow that backs up critical repositories daily.

In [ ]:
# TODO: Create a backup workflow that:
# 1. Identifies repositories tagged as 'critical'
# 2. Creates compressed backups
# 3. Uploads to cloud storage
# 4. Rotates old backups (keep last 7 days)
# 5. Sends notification on completion

backup_workflow = {
    "name": "daily-backup",
    # Your implementation here
}

# Test your workflow
# simulate_workflow_execution(backup_workflow)

### Exercise 2: Implement Error Handling
Add comprehensive error handling to a workflow.

In [ ]:
# TODO: Enhance this workflow with error handling:
# 1. Add retry logic for network operations
# 2. Implement rollback on failure
# 3. Add alerting for critical failures
# 4. Create error recovery steps

workflow_with_errors = {
    "name": "robust-workflow",
    "steps": [
        # Add your steps with error handling
    ]
}

### Exercise 3: Performance Optimization
Optimize a workflow for better performance.

In [ ]:
# TODO: Optimize this workflow:
# 1. Identify steps that can run in parallel
# 2. Add caching where appropriate
# 3. Implement batch processing
# 4. Add resource limits

# Original (slow) workflow
slow_workflow = {
    "name": "slow-workflow",
    "steps": [
        {"name": "scan_repo_1", "action": "scan"},
        {"name": "scan_repo_2", "action": "scan"},
        {"name": "scan_repo_3", "action": "scan"},
        {"name": "analyze_1", "action": "analyze"},
        {"name": "analyze_2", "action": "analyze"},
        {"name": "analyze_3", "action": "analyze"},
        {"name": "report", "action": "generate_report"}
    ]
}

# Your optimized version
optimized_workflow = {
    "name": "optimized-workflow",
    # Your implementation here
}

## Cleanup

In [ ]:
# Clean up workspace
import shutil
if 'workspace' in locals() and os.path.exists(workspace):
    shutil.rmtree(workspace)
    print(f"Cleaned up workspace: {workspace}")

## Summary

In this notebook, you learned:
- Core workflow concepts and components
- YAML syntax for workflow definition
- Creating maintenance and release pipelines
- Conditional execution and control flow
- Parallel vs sequential execution strategies
- Monitoring and debugging workflows
- Advanced patterns for robust automation

## Next Steps

- **Notebook 4**: Advanced Integrations - Combine workflows with clustering
- **Notebook 5**: Data Analysis and Visualization

## Key Takeaways

1. Workflows automate complex multi-step processes
2. Conditional logic enables smart automation
3. Parallel execution significantly improves performance
4. Proper monitoring is essential for production workflows
5. Error handling and recovery make workflows robust
6. Advanced patterns solve common automation challenges